In [1]:
import os
import zipfile
import shutil
import torch
from ultralytics import YOLO
import yaml
from pathlib import Path

# Set up paths
BASE_DIR = r"E:\shittymikshitson"
EXTRACT_DIR = os.path.join(BASE_DIR, "extracted_datasets")
RUNS_DIR = os.path.join(BASE_DIR, "runs")
CHECKPOINT_DIR = os.path.join(BASE_DIR, "checkpoints")

# Create directories
os.makedirs(EXTRACT_DIR, exist_ok=True)
os.makedirs(RUNS_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# List of dataset zip files
dataset_zips = [
    "weapon-detection.v1i.yolov8.zip",
    "weapon-detection.v4-surveillance-perspective.yolov8.zip",
    "weapon-detection.v5-merged-with-pistols-dataset.yolov8.zip",
    "weapon-detection.v13-low-level.yolov8.zip",
    "weapon-detection.v16-remapped-train-80-val-20.yolov8.zip"
]

# Function to extract and analyze datasets
def extract_and_analyze_datasets():
    datasets_info = {}

    for zip_file in dataset_zips:
        zip_path = os.path.join(BASE_DIR, zip_file)
        if not os.path.exists(zip_path):
            print(f"Warning: {zip_file} not found, skipping.")
            continue

        # Create extraction directory for this dataset
        dataset_name = os.path.splitext(zip_file)[0]
        extract_path = os.path.join(EXTRACT_DIR, dataset_name)
        os.makedirs(extract_path, exist_ok=True)

        print(f"Extracting {zip_file}...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

        # Find the data.yaml file
        yaml_files = list(Path(extract_path).glob("**/data.yaml"))
        if not yaml_files:
            yaml_files = list(Path(extract_path).glob("**/*.yaml"))

        if yaml_files:
            yaml_path = str(yaml_files[0])
            print(f"Found YAML file: {yaml_path}")

            # Load and analyze the YAML file
            with open(yaml_path, 'r') as f:
                yaml_data = yaml.safe_load(f)

            # Get class names - handle both list and dict formats
            names = yaml_data.get('names', {})
            num_classes = len(names) if names else 0

            # Store dataset information
            datasets_info[dataset_name] = {
                'yaml_path': yaml_path,
                'classes': names,
                'num_classes': num_classes,
                'path': yaml_data.get('path', extract_path)
            }

            # Update the path in the YAML file to the correct extract path
            parent_dir = os.path.dirname(yaml_path)
            yaml_data['path'] = parent_dir  # Set to the directory containing data.yaml

            with open(yaml_path, 'w') as f:
                yaml.dump(yaml_data, f)

            print(f"Dataset {dataset_name} contains {num_classes} classes")

            # Handle both list and dict formats for classes
            if isinstance(names, dict):
                for idx, class_name in names.items():
                    print(f"  - Class {idx}: {class_name}")
            elif isinstance(names, list):
                for idx, class_name in enumerate(names):
                    print(f"  - Class {idx}: {class_name}")

        else:
            print(f"Warning: No YAML file found in {dataset_name}")

    return datasets_info

# Function to get GPU info
def get_gpu_info():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        gpu_name = torch.cuda.get_device_name(0)
        vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"GPU: {gpu_name}")
        print(f"VRAM: {vram_gb:.2f} GB")
        return device, vram_gb
    else:
        print("No GPU detected. Using CPU.")
        return torch.device("cpu"), 0

# Main function for training
def train_weapon_detector(dataset_info):
    # Set device
    device, vram_gb = get_gpu_info()

    # Iterate through each dataset and train the model
    for dataset_name, info in dataset_info.items():
        yaml_path = info['yaml_path']

        print(f"\nTraining on dataset: {dataset_name}")
        print(f"Using YAML configuration: {yaml_path}")

        # Configure model based on available VRAM
        print("\n=== Setting up model for 3060Ti with 8GB VRAM ===")
        model = YOLO("yolov8s.pt")  # Small model - good balance for weapon detection on 3060Ti
        batch_size = 8  # Reduced batch size for lower memory usage
        print(f"Selected model: YOLOv8s")
        print(f"Batch size: {batch_size}")

        # Check for existing checkpoint
        checkpoint_path = os.path.join(CHECKPOINT_DIR, f"{dataset_name}.pt")
        if os.path.exists(checkpoint_path):
            print(f"Resuming training from checkpoint: {checkpoint_path}")
            model.load(checkpoint_path)

        # Configure hyperparameters
        hyperparams = {
            "data": yaml_path,
            "epochs": 100,
            "imgsz": 640,
            "batch": batch_size,
            "device": device,
            "amp": True,  # Mixed precision for better GPU utilization
            "workers": 2,  # Reduced for stability
            "patience": 20,
            "cos_lr": True,
            "project": RUNS_DIR,
            "name": f"weapon_detection_{dataset_name}",
            "save": True,
            "save_period": 10,
            "exist_ok": True,
            "pretrained": True,
            "optimizer": "AdamW",
            "lr0": 0.001,
            "box": 7.5,  # Box loss gain - higher for better weapon detection
        }

        print("\n=== Starting Training ===")
        try:
            model.train(**hyperparams)
            # Save checkpoint after training
            model.save(checkpoint_path)
        except Exception as e:
            print(f"Error during training on dataset {dataset_name}: {e}")
            continue

        print("\n=== Training Complete ===")
        print("\n=== Running Validation ===")
        try:
            model.val()
        except Exception as e:
            print(f"Error during validation on dataset {dataset_name}: {e}")
            continue

        print("\n=== Exporting Model ===")
        try:
            export_path = os.path.join(RUNS_DIR, f"weapon_detection_{dataset_name}", "weights", "best")
            model.export(format="onnx", dynamic=True)
        except Exception as e:
            print(f"Error during export on dataset {dataset_name}: {e}")
            continue

        print(f"\nTraining on dataset {dataset_name} completed!")
        print(f"Results saved to: {os.path.join(RUNS_DIR, f'weapon_detection_{dataset_name}')}")

if __name__ == "__main__":
    print("=== Weapon Detection System Setup ===")
    print("Analyzing available datasets...")
    datasets = extract_and_analyze_datasets()

    if datasets:
        train_weapon_detector(datasets)
    else:
        print("No valid datasets found! Please check your zip files.")

=== Weapon Detection System Setup ===
Analyzing available datasets...
Extracting weapon-detection.v1i.yolov8.zip...
Found YAML file: E:\shittymikshitson\extracted_datasets\weapon-detection.v1i.yolov8\data.yaml
Dataset weapon-detection.v1i.yolov8 contains 1 classes
  - Class 0: weapon
Extracting weapon-detection.v4-surveillance-perspective.yolov8.zip...
Found YAML file: E:\shittymikshitson\extracted_datasets\weapon-detection.v4-surveillance-perspective.yolov8\data.yaml
Dataset weapon-detection.v4-surveillance-perspective.yolov8 contains 1 classes
  - Class 0: weapon
Extracting weapon-detection.v5-merged-with-pistols-dataset.yolov8.zip...
Found YAML file: E:\shittymikshitson\extracted_datasets\weapon-detection.v5-merged-with-pistols-dataset.yolov8\data.yaml
Dataset weapon-detection.v5-merged-with-pistols-dataset.yolov8 contains 1 classes
  - Class 0: weapon
Extracting weapon-detection.v13-low-level.yolov8.zip...
Found YAML file: E:\shittymikshitson\extracted_datasets\weapon-detection.v13

train: Scanning E:\shittymikshitson\extracted_datasets\weapon-detection.v1i.yolov8\train\labels.cache... 460 images, 163 backgrounds, 0 corrupt: 100%|██████████| 460/460 [00:00<?, ?it/s]
val: Scanning E:\shittymikshitson\extracted_datasets\weapon-detection.v1i.yolov8\valid\labels.cache... 74 images, 1 backgrounds, 0 corrupt: 100%|██████████| 74/74 [00:00<?, ?it/s]


Plotting labels to E:\shittymikshitson\runs\weapon_detection_weapon-detection.v1i.yolov8\labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to E:\shittymikshitson\runs\weapon_detection_weapon-detection.v1i.yolov8
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      1.89G      1.059      1.036      1.239          6        640: 100%|██████████| 58/58 [00:08<00:00,  7.00it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  6.89it/s]

                   all         74         82      0.577      0.183      0.186     0.0786


...

      2/100      2.05G      1.132     0.8313      1.277         12        640: 100%|██████████| 265/265 [00:27<00:00,  9.72it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:01<00:00, 10.09it/s]

                   all        245        256      0.926      0.738      0.769      0.469



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      3/100      2.05G      1.112     0.8246      1.267         14        640:  73%|███████▎  | 193/265 [00:19<00:07,  9.95it/s]


KeyboardInterrupt: 

In [1]:
from ultralytics import YOLO
import os
import time

# Path to the model
MODEL_PATH = r"E:\shittymikshitson\runs\weapon_detection\weights\best.pt"

print(f"Starting ONNX export with troubleshooting settings...")
start_time = time.time()

# Load model
model = YOLO(MODEL_PATH)

# Export with optimized settings for speed
try:
    # Use a fixed size (rather than dynamic) to simplify and speed up conversion
    model.export(
        format="onnx",
        imgsz=640,  # Fixed image size
        simplify=True,  # Simplify the model graph
        opset=13,  # Try older opset which is often more stable
        half=False,  # Use FP32 precision (more compatible)
        dynamic=False,  # Fixed input size for faster export
        verbose=True  # Show detailed progress
    )

    print(f"\nExport completed in {time.time() - start_time:.2f} seconds!")
    print(f"ONNX model saved at: {os.path.splitext(MODEL_PATH)[0]}.onnx")

except Exception as e:
    print(f"Export failed with error: {e}")

    print("\nTrying alternative export settings...")
    # Try alternative settings if the first attempt fails
    try:
        model.export(
            format="onnx",
            opset=11,  # Try even older opset version
            simplify=True,
            verbose=True
        )
        print(f"\nAlternative export completed in {time.time() - start_time:.2f} seconds!")
    except Exception as e2:
        print(f"Alternative export also failed: {e2}")
        print("\nRecommendation: Try updating ultralytics package with 'pip install -U ultralytics'")

Starting ONNX export with troubleshooting settings...
Ultralytics 8.3.99  Python-3.11.9 torch-2.0.1+cu117 CPU (AMD Ryzen 5 5600 6-Core Processor)
Model summary (fused): 72 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from 'E:\shittymikshitson\runs\weapon_detection\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 7, 8400) (64.1 MB)

ONNX: starting export with onnx 1.17.0 opset 13...
============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: slimming with onnxslim 0.1.49...
ONNX: export success  2.6s, saved as 'E:\shittymikshitson\runs\weapon_detection\weights\best.onnx' (42.7 MB)

Export complete (3.2s)
Results saved to E:\shittymikshitson\runs\weapon_detection\weights
Predict:         yolo predict task=detect model=E:\shittymikshitson\runs\weapon_detection\weights\best.onnx imgsz